In [127]:
import sys, os, time

import open3d as o3d
import trimesh
import openmesh as om

import cv2

import scipy as sp
from scipy.sparse.linalg import eigs
from scipy.sparse import csr_matrix, coo_matrix, identity, vstack
from scipy.sparse.linalg import spsolve, lsqr
from scipy.spatial import Delaunay, Voronoi

import numpy as np

import matplotlib.pyplot as plt

from tqdm import tqdm

%matplotlib inline

In [128]:
# Tests of Openmesh
# https://gitlab.vci.rwth-aachen.de:9000/OpenMesh/openmesh-python/tree/master/tests

In [146]:
def read_mesh_om(path_in_models):
    return om.read_trimesh(os.path.join('..', 'models', path_in_models))
def write_mesh_om(mesh, path_in_models):
    om.write_mesh(os.path.join('..', 'models', path_in_models), mesh, vertex_color=True)
def show_mesh_o3d(plys):
    o3d.visualization.draw_geometries(plys)
def write_mesh_o3d(path, mesh):
    o3d.io.write_triangle_mesh(path, mesh)
def read_mesh_o3d(mesh_fp):
    return o3d.io.read_triangle_mesh(mesh_fp)
def read_mesh_trimesh(path_in_models):
    return trimesh.load(os.path.join('..', 'models', path_in_models))
def get_vertices_from_trimesh(mesh):
    return np.asarray(mesh.vertices)
def get_vertice_normals_from_trimesh(mesh):
    return np.asarray(mesh.vertex_normals)

### Calculate Voronoi Poles

In [130]:
INFINITY = 1e9
EPS = 1e-9
def calculate_voronoi_poles(mesh):
    
    vertices = get_vertices_from_trimesh(mesh)
    vertice_normals = get_vertice_normals_from_trimesh(mesh)

    print("Calculating Voronoi Diagram.")
    vor = Voronoi(vertices)
    vor_centers = vor.vertices
    cells = vor.regions
    cell_indices = vor.point_region

    vertices_total = vertices.shape[0]
    voronoi_poles = np.zeros(vertices.shape)
    print("Calculating the Voronoi Pole for each vertex.")
    for vi in tqdm(range(vertices_total)):
        vor_cell = cells[cell_indices[vi]]
        vertice = vertices[vi]
        vertice_normal = vertice_normals[vi]
        max_neg_proj = INFINITY
        voronoi_pole = None
        for vci in vor_cell:
            vor_center = vor_centers[vci]
            if vci == -1:
                continue
            proj = np.dot(vor_center - vertice, vertice_normal)
            if proj < max_neg_proj:
                max_neg_proj = proj
                voronoi_pole = vor_center
        voronoi_poles[vi] = voronoi_pole

    return voronoi_poles

### Calculate Cotangent Laplace Operator

In [153]:
def trimesh_Generate_Area_List(mesh):

    area_list = np.zeros((len(mesh.vertices)))
    face_count = mesh.faces.shape[0]
    face_area = np.asarray(mesh.area_faces)
    faces = np.asarray(mesh.faces)
    for i in range(face_count):
        area_list[faces[i]] += face_area[i] / 3
    
    area_list /= np.mean(area_list)
    return area_list


def trimesh_Generate_Laplace_matrix(mesh):
    
    print('Constructing Laplace Matrix.')
    
    vertices_face_indexs = [[0,1,2],[1,0,2],[2,0,1]] 
    laplace_dict = {}
    area_list = trimesh_Generate_Area_List(mesh)
    face_angles = np.asarray(mesh.face_angles)
    faces = np.asarray(mesh.faces)
    face_count = faces.shape[0]
    
    with tqdm(total=face_count) as tbar:
        for face, angles in zip(faces, face_angles):
            for i in range(3):
                current_angle = angles[i]
                v0_index, v1_index, v2_index = face[vertices_face_indexs[i]]
                delta = 0.5 / area_list[v0_index] / np.tan(current_angle)
                if delta > EPS:
                    laplace_dict[(v1_index, v1_index)] = laplace_dict.get((v1_index, v1_index), 0) - delta
                    laplace_dict[(v2_index, v2_index)] = laplace_dict.get((v2_index, v2_index), 0) - delta
                    laplace_dict[(v1_index, v2_index)] = laplace_dict.get((v1_index, v2_index), 0) + delta
                    laplace_dict[(v2_index, v1_index)] = laplace_dict.get((v2_index, v1_index), 0) + delta
            tbar.update(1)

    # Construct CSR Matrix
    rows, cols = zip(*laplace_dict.keys())
    values = list(laplace_dict.values())
    coo = coo_matrix((values, (rows, cols)), shape=(len(mesh.vertices), len(mesh.vertices)))
    csr = coo.tocsr()

    return csr

### Solve Equation

In [132]:
def construct_Equation(wL, wH, wM, vertices, L, voronoi_poles):
    vn = vertices.shape[0]
    WH_L = wH * identity(vn)
    WM_L = wM * identity(vn)
    A = vstack((wL * L, WH_L, WM_L))
    b = np.vstack((np.zeros((vn, 3)), wH * vertices, wM * voronoi_poles))
    return A, b

### Iteration

In [180]:
it = 5
wL = 0.01
wH = 0.1
wM = 0.2

new_mesh = 'armadillo_0.obj'
mesh = read_mesh_trimesh('armadillo.obj')
voronoi_poles = calculate_voronoi_poles(mesh)

for i in range(it):

    laplace_csr = trimesh_Generate_Laplace_matrix(mesh)

    vertices = get_vertices_from_trimesh(mesh)
    A, b = construct_Equation(wL, wH, wM, vertices, laplace_csr, voronoi_poles)

    new_vertices0 = lsqr(A, b[:, 0])
    new_vertices1 = lsqr(A, b[:, 1])
    new_vertices2 = lsqr(A, b[:, 2])

    new_vertices = np.zeros(vertices.shape)
    new_vertices[:, 0] = new_vertices0[0]
    new_vertices[:, 1] = new_vertices1[0]
    new_vertices[:, 2] = new_vertices2[0]

    mesh.vertices = new_vertices

    write_mesh_o3d('../models/result/' + new_mesh, mesh.as_open3d)
    mesh = read_mesh_trimesh('result/' + new_mesh)
    new_mesh = 'armadillo_%d.obj' % (i + 1)

Calculating Voronoi Diagram.
Calculating the Voronoi Pole for each vertex.


100%|██████████| 25193/25193 [00:01<00:00, 13445.16it/s]


Constructing Laplace Matrix.


100%|██████████| 50382/50382 [00:00<00:00, 65142.93it/s]


Constructing Laplace Matrix.


...

Constructing Laplace Matrix.


100%|██████████| 50382/50382 [00:00<00:00, 65904.66it/s]


Constructing Laplace Matrix.


  0%|          | 0/50382 [00:00<?, ?it/s]/var/folders/m2/_6w5hv4d0xd0bqjjxjdfvxfh0000gn/T/ipykernel_19689/2115848638.py:30: RuntimeWarning: divide by zero encountered in scalar divide
  delta = 0.5 / area_list[v0_index] / np.tan(current_angle)
/var/folders/m2/_6w5hv4d0xd0bqjjxjdfvxfh0000gn/T/ipykernel_19689/2115848638.py:34: RuntimeWarning: invalid value encountered in scalar add
  laplace_dict[(v1_index, v2_index)] = laplace_dict.get((v1_index, v2_index), 0) + delta
100%|██████████| 50382/50382 [00:00<00:00, 67101.83it/s]


ValueError: dimension mismatch

# Testing Blocks

In [176]:
mesh = read_mesh_trimesh('armadillo.obj')

In [178]:
voronoi_poles = calculate_voronoi_poles(mesh)

Calculating Voronoi Diagram.
Calculating the Voronoi Pole for each vertex.


100%|██████████| 25193/25193 [00:01<00:00, 13358.45it/s]


In [177]:
laplace_csr = trimesh_Generate_Laplace_matrix(mesh)

Constructing Laplace Matrix.


100%|██████████| 50382/50382 [00:00<00:00, 59028.61it/s]


In [163]:
laplace = laplace_csr.toarray()

In [164]:
non0idx = np.where(laplace>EPS)
laplace_non0 = laplace[non0idx]
print(np.mean(laplace[non0idx]))

0.6261663245916762


In [155]:
areas = trimesh_Generate_Area_List(mesh)

In [156]:
print(np.mean(areas))

0.9999999999999997


In [152]:
vertices = get_vertices_from_trimesh(mesh)
edges = np.asarray(mesh.edges_unique)
sumlen = 0
for e in edges:
    v0, v1 = vertices[e]
    sumlen += np.linalg.norm(v0 - v1)
print(sumlen/edges.shape[0])

0.011086547901621214


In [181]:
wL = 1
wH = 0.1
wM = 0.2
vertices = get_vertices_from_trimesh(mesh)
A, b = construct_Equation(wL, wH, wM, vertices, laplace_csr, voronoi_poles)

new_vertices0 = lsqr(A, b[:, 0])
new_vertices1 = lsqr(A, b[:, 1])
new_vertices2 = lsqr(A, b[:, 2])

new_vertices = np.zeros(vertices.shape)
new_vertices[:, 0] = new_vertices0[0]
new_vertices[:, 1] = new_vertices1[0]
new_vertices[:, 2] = new_vertices2[0]

mesh.vertices = new_vertices
write_mesh_o3d('../models/result/test.obj', mesh.as_open3d)

ValueError: dimension mismatch